In [ ]:
import os
import sys
from pathlib import Path

import datarobot as dr
from dotenv import load_dotenv

# The notebook should be executed from the project root directory
if "_correct_path" not in locals():
    os.chdir("..")
    sys.path.append(".")
    print(f"changed dir to {Path('.').resolve()})")
    _correct_path = True
load_dotenv()
client = dr.Client()

In [ ]:
import textwrap

from infra.settings_main import project_name
from nbo.custom_metrics import metrics_manager
from nbo.schema import (
    AppDataScienceSettings,
    AppInfraSettings,
    LLMModelSpec,
    OutcomeDetail,
    association_id,
)

custom_metric_baselines = metrics_manager.get_baseline_values()

In [ ]:
app_ds_settings = AppDataScienceSettings(
    page_title="製品トラブル予測オペレーター",
    page_subtitle=textwrap.dedent("""\
        予測AIと生成AIを組み合わせ、製品トラブルを検知します。
        ドロップダウンから選択して送信するだけです！"""),
    record_identifier={
        "column_name": "製造ID",
        "display_name": "製造ID",
    },
    text_explanation_feature="",  # Optional, include when a text variable is present and ngrams feature explanations are desired
    no_text_gen_label=None,  # Optional, include in predictions where an email is not desired
    default_number_of_explanations=3,
    target_probability_description="製造トラブルの発生可能性",  # Preceeded in prompt by "Feature is increasing/decreasing"
    email_prompt=textwrap.dedent("""\
        製造現場のオペレーターが製造パラメータを調整するときに、指定したパラメータで製造トラブルのリスクがあるかどうか参考となる情報をもとめています。
        AIモデルが予測した製造ID{selected_record}の製造トラブルの発生可能性は {prediction_label}であることを伝えてください。
        この文章のトーンは {tone}にしてください。長さは{verbosity}にしてください。
        予測の根拠として、以下の要因を考慮に入れてください。:

        {rsp}"""),
    outcome_details=[  # Each item should be a target value with it's corresponding label and optional description
        OutcomeDetail(
            prediction=0,
            label="製造トラブルの可能性が低い",
            description="製造トラブルの可能性は低いため製造ラインでの製造が許容される",
        ),
        OutcomeDetail(
            prediction=1,
            label="製造トラブルの可能性が高い",
            description="製造トラブルの可能性が高いためパラメータの設定検討が必要",
        ),
    ],
    custom_metric_baselines=custom_metric_baselines,
    association_id_column_name=association_id,
    tones=[
        "専門的",
        "情報に富んでいる",
        "フォーマル",
        "親しみやすい、カジュアル",
        "明るく、面白い",
        "ウィットに富み、遊び心がある",
    ],
    verbosity=["短い", "普通", "長い"],
    system_prompt=textwrap.dedent("""\
        あなたは、製造オペレーターが製造パラメーターを設定する際に、参考情報を与えるスペシャルアドバイザーです。
        あなたの仕事は、製造トラブルの発生確率をもとに、製造パラメータを変更すべきかどうか決定することです。
        あなたは、製造トラブルの発生可能性に関する詳細な情報を持っています。
        また、製造トラブルの発生可能性を決定する要因のリストも持っています。
        あなたの役割は、製造トラブルの発生可能性の情報とその根拠を明確に提示し、パラメータ設定するオペレーターに参考情報を提供することです。
        要因はリスト形式で視覚的に分かりやすく表示してください。
        オペレーターは日本人なので、回答は日本語で生成してください。
        """),
    model_spec=LLMModelSpec(
        input_price_per_1k_tokens=0.001,  # update these values based on the model's pricing
        output_price_per_1k_tokens=0.002,
    ),
)

In [ ]:
from datarobot_pulumi_utils.schema.training import (
    AdvancedOptionsArgs,
    AnalyzeAndModelArgs,
    AutopilotRunArgs,
)

use_case_name = f"NBO Underwriting [{project_name}]"
use_case_description = "Loan Underwriting"

dataset_name = f"NBO Loan Underwriting Training Data [{project_name}]"
file_path = "assets/bleedout_train.csv"


autopilotrun_args = AutopilotRunArgs(
    name=f"NBO Underwriting AutoPilot Run [{project_name}]",
    analyze_and_model_config=AnalyzeAndModelArgs(
        metric="LogLoss", mode="quick", target="ブリードアウト", positive_class=1
    ),
    advanced_options_config=AdvancedOptionsArgs(
        seed=42,
        shap_only_mode=True,
    ),
)

registered_model_name = f"NBO [{project_name}]"

In [ ]:
from datarobotx.idp.autopilot import get_or_create_autopilot_run
from datarobotx.idp.datasets import get_or_create_dataset_from_file
from datarobotx.idp.registered_model_versions import (
    get_or_create_registered_leaderboard_model_version,
)
from datarobotx.idp.use_cases import get_or_create_use_case

In [ ]:
print(f"Creating Use Case {use_case_name}")

if "DATAROBOT_DEFAULT_USE_CASE" in os.environ:
    use_case_id = os.environ["DATAROBOT_DEFAULT_USE_CASE"]
else:
    use_case_id = get_or_create_use_case(
        endpoint=client.endpoint,
        token=client.token,
        name=use_case_name,
        description=use_case_description,
    )

In [ ]:
print(f"Creating Dataset {dataset_name}")
dataset_id = get_or_create_dataset_from_file(
    token=client.token,
    endpoint=client.endpoint,
    name=dataset_name,
    file_path=file_path,
    use_cases=use_case_id,
)

In [ ]:
print(f"Creating Autopilot Run {autopilotrun_args.name}")
project_id = get_or_create_autopilot_run(
    token=client.token,
    endpoint=client.endpoint,
    dataset_id=dataset_id,
    use_case=use_case_id,
    **autopilotrun_args.model_dump(mode="json"),
)

In [ ]:
recommended_model_id = dr.ModelRecommendation.get(project_id).model_id  # type: ignore[union-attr,attr-defined]

try:
    model = dr.Model.get(project_id, recommended_model_id)  # type: ignore[attr-defined]
    prediction_threshold = model.get_roc_curve(
        source="validation"
    ).get_best_f1_threshold()
except Exception:
    prediction_threshold = None

In [ ]:
import pandas as pd

print(f"Best model: {model.model_type}\n\nMetrics:")

pd.DataFrame.from_records(model.metrics)

In [ ]:
print("Creating Registered Model Version...")
registered_model_version_id = get_or_create_registered_leaderboard_model_version(
    token=client.token,
    endpoint=client.endpoint,
    model_id=recommended_model_id,
    registered_model_name=registered_model_name,
    prediction_threshold=prediction_threshold,
)

In [ ]:
app_infra_settings = AppInfraSettings(
    registered_model_name=registered_model_name,
    registered_model_version_id=registered_model_version_id,
    scoring_dataset_id=dataset_id,
    use_case_id=use_case_id,
    project_id=project_id,
)

In [ ]:
import yaml

from infra.settings_main import (
    model_training_output_ds_settings,
    model_training_output_infra_settings,
)

with open(model_training_output_ds_settings, "w") as f:
    yaml.safe_dump(app_ds_settings.model_dump(mode="json"), f, allow_unicode=True)
with open(model_training_output_infra_settings, "w") as f:
    yaml.safe_dump(app_infra_settings.model_dump(mode="json"), f, allow_unicode=True)